# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
weather_df = pd.read_csv("cities.csv")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.08,-159.32,80.60,69,75,13.87,US,1600439033
1,Kruisfontein,-34.00,24.73,66.00,67,100,11.99,ZA,1600439034
2,Okha,53.57,142.95,51.69,92,100,27.31,RU,1600439034
3,Thompson,55.74,-97.86,30.20,100,75,2.37,CA,1600439034
4,Ushuaia,-54.80,-68.30,30.20,100,90,35.57,AR,1600439034


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#Configure gmaps. Use the Lat and Lng as locations and Humidity as the weight.
gmaps.configure(api_key = gkey)
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)


# Plot Heatmap
layout_size = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create Heatmap layer
fig = gmaps.figure(layout = layout_size, map_type = "HYBRID", zoom_level = 2, center=(15,25))

heat_layer = gmaps.heatmap_layer(locations, humidity, 
                                 dissipating = False, max_intensity = 10,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.
weather_conditions_df = weather_df.loc[(weather_df["Humidity"] <= 90) & (weather_df["Wind Speed"] <= 10) & 
                                   (weather_df["Cloudiness"] == 0) & (weather_df["Max Temp"] >= 70) & 
                                   (weather_df["Max Temp"] <= 80)].dropna()

weather_conditions_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,Cidreira,-30.18,-50.21,70.61,74,0,2.95,BR,1600439034
33,Muros,42.78,-9.06,73.00,67,0,7.00,ES,1600439038
102,Karratha,-20.74,116.85,70.23,85,0,4.68,AU,1600439001
161,Cattolica,43.96,12.74,77.00,57,0,8.05,IT,1600439062
196,Guerrero Negro,27.98,-114.06,77.59,52,0,2.30,MX,1600439067


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.

hotel_df = weather_conditions_df
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,Cidreira,-30.18,-50.21,70.61,74,0,2.95,BR,1600439034,
33,Muros,42.78,-9.06,73.00,67,0,7.00,ES,1600439038,
102,Karratha,-20.74,116.85,70.23,85,0,4.68,AU,1600439001,
161,Cattolica,43.96,12.74,77.00,57,0,8.05,IT,1600439062,
196,Guerrero Negro,27.98,-114.06,77.59,52,0,2.30,MX,1600439067,


In [8]:
# Set parameters to search for hotels with 5000 meters.

params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "resort",
    "key": gkey
}

# Hit the Google Places API for each city's coordinates.

# base url
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# Store the first Hotel result into the DataFrame.

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params = params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [9]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,Cidreira,-30.18,-50.21,70.61,74,0,2.95,BR,1600439034,Pousada Vila Di Italy
33,Muros,42.78,-9.06,73.00,67,0,7.00,ES,1600439038,Apartment Av. Castelao
102,Karratha,-20.74,116.85,70.23,85,0,4.68,AU,1600439001,Latitude20 The Dunes Apartments
161,Cattolica,43.96,12.74,77.00,57,0,8.05,IT,1600439062,Castello di Granarola
196,Guerrero Negro,27.98,-114.06,77.59,52,0,2.30,MX,1600439067,Hotel Los Caracoles


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Plot markers on top of the heatmap.

# Create hotel marker layer
hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
                                
# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig